In [3]:
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import pandas as pd
from PIL import Image
import os
import torch


class LeafDataset(Dataset):
    def __init__(self, df, data_dir, transforms=None):
        self.df = df
        self.data_dir = data_dir
        self.transforms = transforms

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_name = self.df.iloc[idx, 0]
        label = self.df.iloc[idx, 1]

        img_path = os.path.join(self.data_dir, 'train_images', img_name)
        img = Image.open(img_path).resize((224, 224))
        if self.transforms:
            img = self.transforms(img)

        return img, torch.tensor(label, dtype=torch.long)



In [5]:
from torchvision.models import resnet18
import torch.nn as nn
import pandas as pd
from torchvision import transforms
from torch.utils.data import DataLoader
import torch
from sklearn.metrics import accuracy_score

import sys

model = resnet18(pretrained=True)
model.fc = nn.Linear(512, 5)


/home/sonujha/rnd/LeafDiseaseClassification/.venv/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/sonujha/rnd/LeafDiseaseClassification/.venv/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [11]:
class config:

    batch_size = 8
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    lr = 0.01
    MODEL_PATH = "models"
    N_EPOCHS = 2
    
    train_tfms = transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor(),
                                     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
    valid_tfms = transforms.Compose(
        [transforms.Resize((224, 224)), transforms.ToTensor(),])


In [12]:
from tqdm import tqdm
import torch


def train_one_epoch(model, train_dl, valid_dl, criterion, optimizer):
    model.train()
    train_loss, train_acc = 0, 0
    loop = tqdm(train_dl)
    for i, (img, label) in enumerate(loop):
        img = img.to(config.device)
        label = label.to(config.device)
        optimizer.zero_grad()
        output = model(img)
        loss = criterion(output, label)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        train_acc += (torch.argmax(output, dim=1) == label).float().mean().item()
    model.eval()
    valid_loss, valid_acc = 0, 0
    with torch.no_grad():
        for i, (img, label) in enumerate(valid_dl):
            img = img.to(config.device)
            label = label.to(config.device)
            output = model(img)
            valid_loss += criterion(output, label).item()
            valid_acc += (torch.argmax(output, dim=1) == label).float().mean().item()
    train_loss = train_loss / len(train_dl)
    valid_loss = valid_loss / len(valid_dl)
    train_acc = train_acc / len(train_dl)
    valid_acc = valid_acc / len(valid_dl)
    print(f"train_loss {train_loss:.3f}, valid_loss {valid_loss:.3f}")
    print(f"train_acc {train_acc:.3f}, valid_acc {valid_acc:.3f}")
    return train_loss, valid_loss

In [13]:
import pandas as pd
from sklearn import model_selection
import os


def get_fold(root):
    csv_path = os.path.join(root, 'train.csv')
    df = pd.read_csv(csv_path)

    df["kfold"] = -1

    df = df.sample(frac=0.1).reset_index(drop=True)

    label = df.label.values

    kf = model_selection.StratifiedKFold(n_splits=5)

    for f, (t_, v_) in enumerate(kf.split(X=df, y=label)):
        df.loc[v_, "kfold"] = f

    #df.to_csv("data/train_fold.csv", index=False)
    return df


In [18]:
from torch.utils.data import DataLoader, Subset
import numpy as np
from pathlib import Path

data_path = '../data'

df = get_fold(data_path)

train_df = df[df.kfold != 0].reset_index(drop=True)
valid_df = df[df.kfold == 1].reset_index(drop=True)

train_ds = LeafDataset(train_df, data_path, transforms=config.train_tfms)
valid_ds = LeafDataset(valid_df, data_path, transforms=config.valid_tfms)

train_ds = Subset(train_ds, np.arange(100))
valid_ds = Subset(valid_ds, np.arange(24))

train_dl = DataLoader(train_ds, batch_size=config.batch_size, shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size=config.batch_size, shuffle=False)

model = model.to(config.device)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=config.lr)

for epoch in range(config.N_EPOCHS):
    train_one_epoch(model, train_dl, valid_dl, loss_fn, optimizer)


100%|█████████████████████████████████████████████████████████| 13/13 [00:05<00:00,  2.42it/s]


train_loss 2.914, valid_loss 412.248
train_acc 0.385, valid_acc 0.125


100%|█████████████████████████████████████████████████████████| 13/13 [00:06<00:00,  1.95it/s]


train_loss 1.806, valid_loss 215.871
train_acc 0.337, valid_acc 0.125


In [19]:
# First, create a hook function to store outputs
activations = {}

def get_activation(name):
    def hook(model, input, output):
        activations[name] = output.detach()
    return hook

# Register hooks for each layer you want to monitor
def register_layer_hooks(model):
    # Register for main layers
    model.layer1.register_forward_hook(get_activation('layer1'))
    model.layer2.register_forward_hook(get_activation('layer2'))
    model.layer3.register_forward_hook(get_activation('layer3'))
    model.layer4.register_forward_hook(get_activation('layer4'))
    
    # Also register for final layer
    model.fc.register_forward_hook(get_activation('fc'))

# In your training/validation loop, you can then examine the activations:
def examine_activations():
    for name, activation in activations.items():
        print(f"\n{name} output:")
        print(f"Shape: {activation.shape}")
        print(f"Mean: {activation.mean().item():.4f}")
        print(f"Std: {activation.std().item():.4f}")
        print(f"Max: {activation.max().item():.4f}")
        print(f"Min: {activation.min().item():.4f}")

In [20]:
# Check input normalization
def check_input(x):
    print(f"Input mean: {x.mean().item():.4f}")
    print(f"Input std: {x.std().item():.4f}")
    print(f"Input max: {x.max().item():.4f}")
    print(f"Input min: {x.min().item():.4f}")

# In pdb:
check_input(inputs)  # Add this before model forward pass

NameError: name 'inputs' is not defined